In [6]:
# 导入包
import itertools
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [7]:
# 导入数据
path_train = 'data_file/train-age-oversampled.csv'

data_train = pd.read_csv(path_train, header=0)
print(data_train.head(5))

   XINGBIE   AGE  HYZK  ZHIYE  ZHICHEN  ZHIWU  XUELI  DWJJLX  DWSSHY   GRJCJS  \
0      1.0  18.0  90.0   90.0    999.0    0.0   99.0   150.0    12.0   1737.0   
1      2.0  35.0  90.0   90.0    999.0    0.0   99.0   110.0     0.0   4894.0   
2      1.0  27.0  90.0   90.0    999.0    0.0   99.0   150.0     9.0  10297.0   
3      1.0  37.0  90.0   90.0    999.0    0.0   99.0   150.0     7.0  10071.5   
4      2.0  33.0  90.0   90.0    999.0    0.0   99.0   900.0    14.0   2007.0   

   GRZHZT      GRZHYE  GRZHSNJZYE  GRZHDNGJYE   GRYJCE     DKFFE        DKYE  \
0     1.0    3223.515     801.310     837.000   312.00  175237.0  154112.935   
1     1.0   18055.195   53213.220    1065.200   795.84  300237.0  298252.945   
2     1.0   27426.600   13963.140    7230.020  1444.20  150237.0  147339.130   
3     1.0  111871.130   99701.265    2271.295  1417.14  350237.0  300653.780   
4     1.0     237.000   11028.875      35.780   325.50  150237.0  145185.010   

    DKLL  label  
0  2.708    0.

In [8]:
# 连续变量
feature_li = ['GRJCJS', 'GRZHYE', 'GRZHSNJZYE', 'GRZHDNGJYE', 'GRYJCE', 'DKFFE', 'DKYE']

# 创造新的变量
# 使用最简单的方法：1 简单数学计算（主要是连续变量之间
def feature_arithmetic(data, f1, f2, operation):
    # 连接两个变量，创造新的变量名
    new_name = f1 +'_' + operation + '_' + f2
    
    if operation == 'add':
        new_df = data.loc[:, f1] + data.loc[:, f2]
    elif operation == 'subtract':
        new_df = data.loc[:, f1] - data.loc[:, f2]
    elif operation == 'multiply':
        new_df = data.loc[:, f1] * data.loc[:, f2]

    # 代表  a / b， 前者 / 后者
    elif operation == 'divide_ab':
        new_df = data.loc[:, f1] / data.loc[:, f2]
        new_df.astype(float)
    # 代表 b / a， 后者 / 前者
    elif operation == 'divide_ba':
        new_df = data.loc[:, f2] / data.loc[:, f1]
        new_df.astype(float)

    return new_name, new_df

# TODO：将feature_li默认参数转变成不变的，这里有风险
def bin_features(data, feature_li=feature_li):
    name_5_operations = ['add', 'subtract', 'multiply', 'divide_ab', 'divide_ba']
    # feature_arithmetic_names = []
    new_combin_li = list(itertools.combinations(feature_li, 2))
    print('len(new_combin_li)', len(new_combin_li)) # for check

    # generate feature using basic math operation(four operations)
    vocant = []
    for f1, f2 in new_combin_li:
        for operation in name_5_operations:
            vocant.append(feature_arithmetic(data, f1, f2, operation))
    
    # the number of feature created.
    print(len(new_combin_li) * len(name_5_operations)) # for check
    
    # add new feature to data
    for new_name, new_df in vocant:
        data[new_name] = new_df

    return data  

# call the function to generate feature
data_feature_bined = bin_features(data_train)

# set label in the last column
df_label = data_feature_bined['label']
data_feature_bined = data_feature_bined.drop('label', axis=1)
data_feature_bined['label'] = df_label

# write to csv file
# also for the correlation matrix in the excel.
data_feature_bined.to_csv('data_file/data_train_generation.csv')
data_feature_bined.corr().to_excel('data_file/data_train_corr.xlsx')

len(new_combin_li) 21
105


In [9]:
# categorial variables
# using correlation method to remain top 50% variables 
# that is correlated with label,
# ignoring the categorial variables
categorial_var = ['XINGBIE', 'AGE', 'HYZK', 'ZHIYE', 'ZHICHEN', 'ZHIWU',
'XUELI', 'DWJJLX', 'DWSSHY', 'GRZHZT', 'DKLL', 'label']
  

def select_top_half_feature(data, categorial_var=categorial_var):
    """1. 计算变量之间的相关性corr，根据label标签排序
    2. 确定需要留下来的特征个数 num_remain
    3. 在2中，去掉分类变量的个数 num_remain - len(categorial_var)
    4. 去掉分类变量，对剩下变量的相关系数排序(因为在排序的时候考虑分类变量)
    5. 找到前面前num_remain - len(categorial_var) 个变量
    6. 按照 分类变量 5中的变量 label 的特征顺序，返回一个dataframe"""
    
    # 1. corr, label排序
    data_corr = data.corr().loc[:, 'label']
    print(len(data_corr))

    # 2.确定需要留下来的变量个数
    num_drop = int(len(data_corr.index) / 2)
    num_remain = len(data_corr.index) - num_drop

    # 3 算作3 >_<
    feature_nocategor = [fea for fea in data_corr.index if fea not in categorial_var]

    # 4 按照相关性，对除去分类变量的数据，进行排序
    feature_rank_abs = pd.DataFrame(
        np.abs(data_corr.loc[feature_nocategor])
        ).sort_values(by='label', ascending=False, inplace=False)
    print('remain features(no categorical)', num_remain - len(categorial_var))
    # 5 除去分类变量，剩下需要的变量
    cols = feature_rank_abs.index.tolist()[:num_remain - len(categorial_var)]

    # 5 6 将label 放在最后一列（非必要）
    categorial_var.remove('label')
    feature_need = categorial_var + cols + ['label']
    return data.loc[:, feature_need]

data_corr_selected = select_top_half_feature(data_feature_bined)

124
remain features(no categorical) 50


In [10]:
"""保存数据
其实后来看的话，只需要保存相关的特征就好了，后面直接调用这个文件的函数，计算出那些需要留下来的特征
避免了过多的文件读写操作
index=0 是说不将索引（0, 1, 2, 3...) 写进csv文件中"""
data_corr_selected.to_csv('data_file/data_train_corr_selected.csv', index=0)